In [1]:
import os
import csv
import time
import datetime
import random
import json

import warnings
from collections import Counter
from math import sqrt

import gensim
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score
warnings.filterwarnings("ignore")

In [2]:
# 配置参数

class TrainingConfig(object):
    epoches = 10
    evaluateEvery = 100
    checkpointEvery = 100
    learningRate = 0.001
    
    
class ModelConfig(object):
    embeddingSize = 200
    
    filters = 128  # 内层一维卷积核的数量，外层卷积核的数量应该等于embeddingSize，因为要确保每个layer后的输出维度和输入维度是一致的。
    numHeads = 8  # Attention 的头数
    numBlocks = 1  # 设置transformer block的数量
    epsilon = 1e-8  # LayerNorm 层中的最小除数
    keepProp = 0.9  # multi head attention 中的dropout
    
    dropoutKeepProb = 0.5 # 全连接层的dropout
    l2RegLambda = 0.0
    
    
class Config(object):
    sequenceLength = 200  # 取了所有序列长度的均值
    batchSize = 128
    
    dataSource = "../data/preProcess/labeledTrain.csv"
    
    stopWordSource = "../data/english"
    
    numClasses = 2
    
    rate = 0.8  # 训练集的比例
    
    training = TrainingConfig()
    
    model = ModelConfig()

    
# 实例化配置参数对象
config = Config()

In [3]:
# 数据预处理的类，生成训练集和测试集

class Dataset(object):
    def __init__(self, config):
        self._dataSource = config.dataSource
        self._stopWordSource = config.stopWordSource  
        
        self._sequenceLength = config.sequenceLength  # 每条输入的序列处理为定长
        self._embeddingSize = config.model.embeddingSize
        self._batchSize = config.batchSize
        self._rate = config.rate
        
        self._stopWordDict = {}
        
        self.trainReviews = []
        self.trainLabels = []
        
        self.evalReviews = []
        self.evalLabels = []
        
        self.wordEmbedding =None
        
        self._wordToIndex = {}
        self._indexToWord = {}
        
    def _readData(self, filePath):
        """
        从csv文件中读取数据集
        """
        
        df = pd.read_csv(filePath)
        labels = df["sentiment"].tolist()
        review = df["review"].tolist()
        reviews = [line.strip().split() for line in review]

        return reviews, labels

    def _reviewProcess(self, review, sequenceLength, wordToIndex):
        """
        将数据集中的每条评论用index表示
        wordToIndex中“pad”对应的index为0
        """
        
        reviewVec = np.zeros((sequenceLength))
        sequenceLen = sequenceLength
        
        # 判断当前的序列是否小于定义的固定序列长度
        if len(review) < sequenceLength:
            sequenceLen = len(review)
            
        for i in range(sequenceLen):
            if review[i] in wordToIndex:
                reviewVec[i] = wordToIndex[review[i]]
            else:
                reviewVec[i] = wordToIndex["UNK"]

        return reviewVec

    def _genTrainEvalData(self, x, y, rate):
        """
        生成训练集和验证集
        """
        
        reviews = []
        labels = []
        
        # 遍历所有的文本，将文本中的词转换成index表示
        for i in range(len(x)):
            reviewVec = self._reviewProcess(x[i], self._sequenceLength, self._wordToIndex)
            reviews.append(reviewVec)
            
            labels.append([y[i]])
            
        trainIndex = int(len(x) * rate)
        
        trainReviews = np.asarray(reviews[:trainIndex], dtype="int64")
        trainLabels = np.array(labels[:trainIndex], dtype="float32")
        
        evalReviews = np.asarray(reviews[trainIndex:], dtype="int64")
        evalLabels = np.array(labels[trainIndex:], dtype="float32")

        return trainReviews, trainLabels, evalReviews, evalLabels
        
    def _genVocabulary(self, reviews):
        """
        生成词向量和词汇-索引映射字典，可以用全数据集
        """
        
        allWords = [word for review in reviews for word in review]
        
        # 去掉停用词
        subWords = [word for word in allWords if word not in self.stopWordDict]
        
        wordCount = Counter(subWords)  # 统计词频
        sortWordCount = sorted(wordCount.items(), key=lambda x: x[1], reverse=True)
        
        # 去除低频词
        words = [item[0] for item in sortWordCount if item[1] >= 5]
        
        vocab, wordEmbedding = self._getWordEmbedding(words)
        self.wordEmbedding = wordEmbedding
        
        self._wordToIndex = dict(zip(vocab, list(range(len(vocab)))))
        self._indexToWord = dict(zip(list(range(len(vocab))), vocab))
        
        # 将词汇-索引映射表保存为json数据，之后做inference时直接加载来处理数据
        with open("../data/wordJson/wordToIndex.json", "w", encoding="utf-8") as f:
            json.dump(self._wordToIndex, f)
        
        with open("../data/wordJson/indexToWord.json", "w", encoding="utf-8") as f:
            json.dump(self._indexToWord, f)
            
    def _getWordEmbedding(self, words):
        """
        按照我们的数据集中的单词取出预训练好的word2vec中的词向量
        """
        
        wordVec = gensim.models.KeyedVectors.load_word2vec_format("../word2vec/word2Vec.bin", binary=True)
        vocab = []
        wordEmbedding = []
        
        # 添加 "pad" 和 "UNK", 
        vocab.append("pad")
        vocab.append("UNK")
        wordEmbedding.append(np.zeros(self._embeddingSize))
        wordEmbedding.append(np.random.randn(self._embeddingSize))
        
        for word in words:
            try:
                vector = wordVec.wv[word]
                vocab.append(word)
                wordEmbedding.append(vector)
            except:
                print(word + "不存在于词向量中")
                
        return vocab, np.array(wordEmbedding)
    
    def _readStopWord(self, stopWordPath):
        """
        读取停用词
        """
        
        with open(stopWordPath, "r") as f:
            stopWords = f.read()
            stopWordList = stopWords.splitlines()
            # 将停用词用列表的形式生成，之后查找停用词时会比较快
            self.stopWordDict = dict(zip(stopWordList, list(range(len(stopWordList)))))
            
    def dataGen(self):
        """
        初始化训练集和验证集
        """
        
        # 初始化停用词
        self._readStopWord(self._stopWordSource)
        
        # 初始化数据集
        reviews, labels = self._readData(self._dataSource)
        
        # 初始化词汇-索引映射表和词向量矩阵
        self._genVocabulary(reviews)
        
        # 初始化训练集和测试集
        trainReviews, trainLabels, evalReviews, evalLabels = self._genTrainEvalData(reviews, labels, self._rate)
        self.trainReviews = trainReviews
        self.trainLabels = trainLabels
        
        self.evalReviews = evalReviews
        self.evalLabels = evalLabels
        
        
data = Dataset(config)
data.dataGen()

In [4]:
print("train data shape: {}".format(data.trainReviews.shape))
print("train label shape: {}".format(data.trainLabels.shape))
print("eval data shape: {}".format(data.evalReviews.shape))

train data shape: (20000, 200)
train label shape: (20000, 1)
eval data shape: (5000, 200)


In [5]:
# 输出batch数据集

def nextBatch(x, y, batchSize):
        """
        生成batch数据集，用生成器的方式输出
        """
    
        perm = np.arange(len(x))
        np.random.shuffle(perm)
        x = x[perm]
        y = y[perm]
        
        numBatches = len(x) // batchSize

        for i in range(numBatches):
            start = i * batchSize
            end = start + batchSize
            batchX = np.array(x[start: end], dtype="int64")
            batchY = np.array(y[start: end], dtype="float32")
            
            yield batchX, batchY

In [6]:
# 生成位置嵌入
def fixedPositionEmbedding(batchSize, sequenceLen):
    embeddedPosition = []
    for batch in range(batchSize):
        x = []
        for step in range(sequenceLen):
            a = np.zeros(sequenceLen)
            a[step] = 1
            x.append(a)
        embeddedPosition.append(x)
    
    return np.array(embeddedPosition, dtype="float32")

In [7]:
# 模型构建

class Transformer(object):
    """
    Transformer Encoder 用于文本分类
    """
    def __init__(self, config, wordEmbedding):

        # 定义模型的输入
        self.inputX = tf.placeholder(tf.int32, [None, config.sequenceLength], name="inputX")
        self.inputY = tf.placeholder(tf.float32, [None, 1], name="inputY")
        
        self.dropoutKeepProb = tf.placeholder(tf.float32, name="dropoutKeepProb")
        self.embeddedPosition = tf.placeholder(tf.float32, [None, config.sequenceLength, config.sequenceLength], name="embeddedPosition")
        
        self.config = config
        
        # 定义l2损失
        l2Loss = tf.constant(0.0)
        
        # 词嵌入层, 位置向量的定义方式有两种：一是直接用固定的one-hot的形式传入，然后和词向量拼接，在当前的数据集上表现效果更好。另一种
        # 就是按照论文中的方法实现，这样的效果反而更差，可能是增大了模型的复杂度，在小数据集上表现不佳。
        
        with tf.name_scope("embedding"):

            # 利用预训练的词向量初始化词嵌入矩阵
            self.W = tf.Variable(tf.cast(wordEmbedding, dtype=tf.float32, name="word2vec") ,name="W")
            # 利用词嵌入矩阵将输入的数据中的词转换成词向量，维度[batch_size, sequence_length, embedding_size]
            self.embedded = tf.nn.embedding_lookup(self.W, self.inputX)
            self.embeddedWords = tf.concat([self.embedded, self.embeddedPosition], -1)

        with tf.name_scope("transformer"):
            for i in range(config.model.numBlocks):
                with tf.name_scope("transformer-{}".format(i + 1)):
            
                    # 维度[batch_size, sequence_length, embedding_size]
                    multiHeadAtt = self._multiheadAttention(rawKeys=self.inputX, queries=self.embeddedWords,
                                                            keys=self.embeddedWords)
                    # 维度[batch_size, sequence_length, embedding_size]
                    self.embeddedWords = self._feedForward(multiHeadAtt, 
                                                           [config.model.filters, config.model.embeddingSize + config.sequenceLength])
                
            outputs = tf.reshape(self.embeddedWords, [-1, config.sequenceLength * (config.model.embeddingSize + config.sequenceLength)])

        outputSize = outputs.get_shape()[-1].value

#         with tf.name_scope("wordEmbedding"):
#             self.W = tf.Variable(tf.cast(wordEmbedding, dtype=tf.float32, name="word2vec"), name="W")
#             self.wordEmbedded = tf.nn.embedding_lookup(self.W, self.inputX)
        
#         with tf.name_scope("positionEmbedding"):
#             print(self.wordEmbedded)
#             self.positionEmbedded = self._positionEmbedding()
            
#         self.embeddedWords = self.wordEmbedded + self.positionEmbedded
            
#         with tf.name_scope("transformer"):
#             for i in range(config.model.numBlocks):
#                 with tf.name_scope("transformer-{}".format(i + 1)):
            
#                     # 维度[batch_size, sequence_length, embedding_size]
#                     multiHeadAtt = self._multiheadAttention(rawKeys=self.wordEmbedded, queries=self.embeddedWords,
#                                                             keys=self.embeddedWords)
#                     # 维度[batch_size, sequence_length, embedding_size]
#                     self.embeddedWords = self._feedForward(multiHeadAtt, [config.model.filters, config.model.embeddingSize])
                
#             outputs = tf.reshape(self.embeddedWords, [-1, config.sequenceLength * (config.model.embeddingSize)])

#         outputSize = outputs.get_shape()[-1].value
        
        with tf.name_scope("dropout"):
            outputs = tf.nn.dropout(outputs, keep_prob=self.dropoutKeepProb)
    
        # 全连接层的输出
        with tf.name_scope("output"):
            outputW = tf.get_variable(
                "outputW",
                shape=[outputSize, 1],
                initializer=tf.contrib.layers.xavier_initializer())
            
            outputB= tf.Variable(tf.constant(0.1, shape=[1]), name="outputB")
            l2Loss += tf.nn.l2_loss(outputW)
            l2Loss += tf.nn.l2_loss(outputB)
            self.predictions = tf.nn.xw_plus_b(outputs, outputW, outputB, name="predictions")
            self.binaryPreds = tf.cast(tf.greater_equal(self.predictions, 0.5), tf.float32, name="binaryPreds")
        
        # 计算二元交叉熵损失
        with tf.name_scope("loss"):
            
            losses = tf.nn.sigmoid_cross_entropy_with_logits(logits=self.predictions, labels=self.inputY)
            self.loss = tf.reduce_mean(losses) + config.model.l2RegLambda * l2Loss
            
    def _layerNormalization(self, inputs, scope="layerNorm"):
        # LayerNorm层和BN层有所不同
        epsilon = self.config.model.epsilon

        inputsShape = inputs.get_shape() # [batch_size, sequence_length, embedding_size]

        paramsShape = inputsShape[-1:]

        # LayerNorm是在最后的维度上计算输入的数据的均值和方差，BN层是考虑所有维度的
        # mean, variance的维度都是[batch_size, sequence_len, 1]
        mean, variance = tf.nn.moments(inputs, [-1], keep_dims=True)

        beta = tf.Variable(tf.zeros(paramsShape))

        gamma = tf.Variable(tf.ones(paramsShape))
        normalized = (inputs - mean) / ((variance + epsilon) ** .5)
        
        outputs = gamma * normalized + beta

        return outputs
            
    def _multiheadAttention(self, rawKeys, queries, keys, numUnits=None, causality=False, scope="multiheadAttention"):
        # rawKeys 的作用是为了计算mask时用的，因为keys是加上了position embedding的，其中不存在padding为0的值
        
        numHeads = self.config.model.numHeads
        keepProp = self.config.model.keepProp
        
        if numUnits is None:  # 若是没传入值，直接去输入数据的最后一维，即embedding size.
            numUnits = queries.get_shape().as_list()[-1]

        # tf.layers.dense可以做多维tensor数据的非线性映射，在计算self-Attention时，一定要对这三个值进行非线性映射，
        # 其实这一步就是论文中Multi-Head Attention中的对分割后的数据进行权重映射的步骤，我们在这里先映射后分割，原则上是一样的。
        # Q, K, V的维度都是[batch_size, sequence_length, embedding_size]
        Q = tf.layers.dense(queries, numUnits, activation=tf.nn.relu)
        K = tf.layers.dense(keys, numUnits, activation=tf.nn.relu)
        V = tf.layers.dense(keys, numUnits, activation=tf.nn.relu)

        # 将数据按最后一维分割成num_heads个, 然后按照第一维拼接
        # Q, K, V 的维度都是[batch_size * numHeads, sequence_length, embedding_size/numHeads]
        Q_ = tf.concat(tf.split(Q, numHeads, axis=-1), axis=0) 
        K_ = tf.concat(tf.split(K, numHeads, axis=-1), axis=0) 
        V_ = tf.concat(tf.split(V, numHeads, axis=-1), axis=0)

        # 计算keys和queries之间的点积，维度[batch_size * numHeads, queries_len, key_len], 后两维是queries和keys的序列长度
        similary = tf.matmul(Q_, tf.transpose(K_, [0, 2, 1]))

        # 对计算的点积进行缩放处理，除以向量长度的根号值
        scaledSimilary = similary / (K_.get_shape().as_list()[-1] ** 0.5)

        # 在我们输入的序列中会存在padding这个样的填充词，这种词应该对最终的结果是毫无帮助的，原则上说当padding都是输入0时，
        # 计算出来的权重应该也是0，但是在transformer中引入了位置向量，当和位置向量相加之后，其值就不为0了，因此在添加位置向量
        # 之前，我们需要将其mask为0。虽然在queries中也存在这样的填充词，但原则上模型的结果之和输入有关，而且在self-Attention中
        # queryies = keys，因此只要一方为0，计算出的权重就为0。
        # 具体关于key mask的介绍可以看看这里： https://github.com/Kyubyong/transformer/issues/3

        # 利用tf，tile进行张量扩张， 维度[batch_size * numHeads, keys_len] keys_len = keys 的序列长度
        keyMasks = tf.tile(rawKeys, [numHeads, 1]) 

        # 增加一个维度，并进行扩张，得到维度[batch_size * numHeads, queries_len, keys_len]
        keyMasks = tf.tile(tf.expand_dims(keyMasks, 1), [1, tf.shape(queries)[1], 1])

        # tf.ones_like生成元素全为1，维度和scaledSimilary相同的tensor, 然后得到负无穷大的值
        paddings = tf.ones_like(scaledSimilary) * (-2 ** (32 + 1))

        # tf.where(condition, x, y),condition中的元素为bool值，其中对应的True用x中的元素替换，对应的False用y中的元素替换
        # 因此condition,x,y的维度是一样的。下面就是keyMasks中的值为0就用paddings中的值替换
        maskedSimilary = tf.where(tf.equal(keyMasks, 0), paddings, scaledSimilary) # 维度[batch_size * numHeads, queries_len, key_len]

        # 在计算当前的词时，只考虑上文，不考虑下文，出现在Transformer Decoder中。在文本分类时，可以只用Transformer Encoder。
        # Decoder是生成模型，主要用在语言生成中
        if causality:
            diagVals = tf.ones_like(maskedSimilary[0, :, :])  # [queries_len, keys_len]
            tril = tf.contrib.linalg.LinearOperatorTriL(diagVals).to_dense()  # [queries_len, keys_len]
            masks = tf.tile(tf.expand_dims(tril, 0), [tf.shape(maskedSimilary)[0], 1, 1])  # [batch_size * numHeads, queries_len, keys_len]

            paddings = tf.ones_like(masks) * (-2 ** (32 + 1))
            maskedSimilary = tf.where(tf.equal(masks, 0), paddings, maskedSimilary)  # [batch_size * numHeads, queries_len, keys_len]

        # 通过softmax计算权重系数，维度 [batch_size * numHeads, queries_len, keys_len]
        weights = tf.nn.softmax(maskedSimilary)

        # 加权和得到输出值, 维度[batch_size * numHeads, sequence_length, embedding_size/numHeads]
        outputs = tf.matmul(weights, V_)

        # 将多头Attention计算的得到的输出重组成最初的维度[batch_size, sequence_length, embedding_size]
        outputs = tf.concat(tf.split(outputs, numHeads, axis=0), axis=2)
        
        outputs = tf.nn.dropout(outputs, keep_prob=keepProp)

        # 对每个subLayers建立残差连接，即H(x) = F(x) + x
        outputs += queries
        # normalization 层
        outputs = self._layerNormalization(outputs)
        return outputs

    def _feedForward(self, inputs, filters, scope="multiheadAttention"):
        # 在这里的前向传播采用卷积神经网络
        
        # 内层
        params = {"inputs": inputs, "filters": filters[0], "kernel_size": 1,
                  "activation": tf.nn.relu, "use_bias": True}
        outputs = tf.layers.conv1d(**params)

        # 外层
        params = {"inputs": outputs, "filters": filters[1], "kernel_size": 1,
                  "activation": None, "use_bias": True}

        # 这里用到了一维卷积，实际上卷积核尺寸还是二维的，只是只需要指定高度，宽度和embedding size的尺寸一致
        # 维度[batch_size, sequence_length, embedding_size]
        outputs = tf.layers.conv1d(**params)

        # 残差连接
        outputs += inputs

        # 归一化处理
        outputs = self._layerNormalization(outputs)

        return outputs
    
    def _positionEmbedding(self, scope="positionEmbedding"):
        # 生成可训练的位置向量
        batchSize = self.config.batchSize
        sequenceLen = self.config.sequenceLength
        embeddingSize = self.config.model.embeddingSize
        
        # 生成位置的索引，并扩张到batch中所有的样本上
        positionIndex = tf.tile(tf.expand_dims(tf.range(sequenceLen), 0), [batchSize, 1])

        # 根据正弦和余弦函数来获得每个位置上的embedding的第一部分
        positionEmbedding = np.array([[pos / np.power(10000, (i-i%2) / embeddingSize) for i in range(embeddingSize)] 
                                      for pos in range(sequenceLen)])

        # 然后根据奇偶性分别用sin和cos函数来包装
        positionEmbedding[:, 0::2] = np.sin(positionEmbedding[:, 0::2])
        positionEmbedding[:, 1::2] = np.cos(positionEmbedding[:, 1::2])

        # 将positionEmbedding转换成tensor的格式
        positionEmbedding_ = tf.cast(positionEmbedding, dtype=tf.float32)

        # 得到三维的矩阵[batchSize, sequenceLen, embeddingSize]
        positionEmbedded = tf.nn.embedding_lookup(positionEmbedding_, positionIndex)

        return positionEmbedded
            

In [8]:
# 定义性能指标函数

def mean(item):
    return sum(item) / len(item)


def genMetrics(trueY, predY, binaryPredY):
    """
    生成acc和auc值
    """
    auc = roc_auc_score(trueY, predY)
    accuracy = accuracy_score(trueY, binaryPredY)
    precision = precision_score(trueY, binaryPredY)
    recall = recall_score(trueY, binaryPredY)
    
    return round(accuracy, 4), round(auc, 4), round(precision, 4), round(recall, 4)

In [ ]:
# 训练模型

# 生成训练集和验证集
trainReviews = data.trainReviews
trainLabels = data.trainLabels
evalReviews = data.evalReviews
evalLabels = data.evalLabels

wordEmbedding = data.wordEmbedding

embeddedPosition = fixedPositionEmbedding(config.batchSize, config.sequenceLength)

# 定义计算图
with tf.Graph().as_default():

    session_conf = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)
    session_conf.gpu_options.allow_growth=True
    session_conf.gpu_options.per_process_gpu_memory_fraction = 0.9  # 配置gpu占用率  

    sess = tf.Session(config=session_conf)
    
    # 定义会话
    with sess.as_default():
        transformer = Transformer(config, wordEmbedding)
        
        globalStep = tf.Variable(0, name="globalStep", trainable=False)
        # 定义优化函数，传入学习速率参数
        optimizer = tf.train.AdamOptimizer(config.training.learningRate)
        # 计算梯度,得到梯度和变量
        gradsAndVars = optimizer.compute_gradients(transformer.loss)
        # 将梯度应用到变量下，生成训练器
        trainOp = optimizer.apply_gradients(gradsAndVars, global_step=globalStep)
        
        # 用summary绘制tensorBoard
        gradSummaries = []
        for g, v in gradsAndVars:
            if g is not None:
                tf.summary.histogram("{}/grad/hist".format(v.name), g)
                tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
        
        outDir = os.path.abspath(os.path.join(os.path.curdir, "summarys"))
        print("Writing to {}\n".format(outDir))
        
        lossSummary = tf.summary.scalar("loss", transformer.loss)
        summaryOp = tf.summary.merge_all()
        
        trainSummaryDir = os.path.join(outDir, "train")
        trainSummaryWriter = tf.summary.FileWriter(trainSummaryDir, sess.graph)
        
        evalSummaryDir = os.path.join(outDir, "eval")
        evalSummaryWriter = tf.summary.FileWriter(evalSummaryDir, sess.graph)
        
        
        # 初始化所有变量
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=5)
        
        # 保存模型的一种方式，保存为pb文件
#         builder = tf.saved_model.builder.SavedModelBuilder("../model/Transformer/savedModel")
        sess.run(tf.global_variables_initializer())

        def trainStep(batchX, batchY):
            """
            训练函数
            """   
            feed_dict = {
              transformer.inputX: batchX,
              transformer.inputY: batchY,
              transformer.dropoutKeepProb: config.model.dropoutKeepProb,
              transformer.embeddedPosition: embeddedPosition
            }
            _, summary, step, loss, predictions, binaryPreds = sess.run(
                [trainOp, summaryOp, globalStep, transformer.loss, transformer.predictions, transformer.binaryPreds],
                feed_dict)
            timeStr = datetime.datetime.now().isoformat()
            acc, auc, precision, recall = genMetrics(batchY, predictions, binaryPreds)
            print("{}, step: {}, loss: {}, acc: {}, auc: {}, precision: {}, recall: {}".format(timeStr, step, loss, acc, auc, precision, recall))
            trainSummaryWriter.add_summary(summary, step)

        def devStep(batchX, batchY):
            """
            验证函数
            """
            feed_dict = {
              transformer.inputX: batchX,
              transformer.inputY: batchY,
              transformer.dropoutKeepProb: 1.0,
              transformer.embeddedPosition: embeddedPosition
            }
            summary, step, loss, predictions, binaryPreds = sess.run(
                [summaryOp, globalStep, transformer.loss, transformer.predictions, transformer.binaryPreds],
                feed_dict)
            
            acc, auc, precision, recall = genMetrics(batchY, predictions, binaryPreds)
            
            evalSummaryWriter.add_summary(summary, step)
            
            return loss, acc, auc, precision, recall
        
        for i in range(config.training.epoches):
            # 训练模型
            print("start training model")
            for batchTrain in nextBatch(trainReviews, trainLabels, config.batchSize):
                trainStep(batchTrain[0], batchTrain[1])

                currentStep = tf.train.global_step(sess, globalStep) 
                if currentStep % config.training.evaluateEvery == 0:
                    print("\nEvaluation:")
                    
                    losses = []
                    accs = []
                    aucs = []
                    precisions = []
                    recalls = []
                    
                    for batchEval in nextBatch(evalReviews, evalLabels, config.batchSize):
                        loss, acc, auc, precision, recall = devStep(batchEval[0], batchEval[1])
                        losses.append(loss)
                        accs.append(acc)
                        aucs.append(auc)
                        precisions.append(precision)
                        recalls.append(recall)
                        
                    time_str = datetime.datetime.now().isoformat()
                    print("{}, step: {}, loss: {}, acc: {}, auc: {}, precision: {}, recall: {}".format(time_str, currentStep, mean(losses), 
                                                                                                       mean(accs), mean(aucs), mean(precisions),
                                                                                                       mean(recalls)))
                    
#                 if currentStep % config.training.checkpointEvery == 0:
#                     # 保存模型的另一种方法，保存checkpoint文件
#                     path = saver.save(sess, "../model/Transformer/model/my-model", global_step=currentStep)
#                     print("Saved model checkpoint to {}\n".format(path))
                    
#         inputs = {"inputX": tf.saved_model.utils.build_tensor_info(transformer.inputX),
#                   "keepProb": tf.saved_model.utils.build_tensor_info(transformer.dropoutKeepProb)}

#         outputs = {"binaryPreds": tf.saved_model.utils.build_tensor_info(transformer.binaryPreds)}

#         prediction_signature = tf.saved_model.signature_def_utils.build_signature_def(inputs=inputs, outputs=outputs,
#                                                                                       method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME)
#         legacy_init_op = tf.group(tf.tables_initializer(), name="legacy_init_op")
#         builder.add_meta_graph_and_variables(sess, [tf.saved_model.tag_constants.SERVING],
#                                             signature_def_map={"predict": prediction_signature}, legacy_init_op=legacy_init_op)

#         builder.save()

INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name dense/kernel:0/grad/hist is illegal; using dense/kernel_0/grad/hist instead.
INFO:tensorflow:Summary name dense/kernel:0/grad/sparsity is illegal; using dense/kernel_0/grad/sparsity instead.
INFO:tensorflow:Summary name dense/bias:0/grad/hist is illegal; using dense/bias_0/grad/hist instead.
INFO:tensorflow:Summary name dense/bias:0/grad/sparsity is illegal; using dense/bias_0/grad/sparsity instead.
INFO:tensorflow:Summary name dense_1/kernel:0/grad/hist is illegal; using dense_1/kernel_0/grad/hist instead.
INFO:tensorflow:Summary name dense_1/kernel:0/grad/sparsity is illegal; using dense_1/kernel_0/grad/sparsity instead.
INFO:tensorflow:Summary name dense_1/bias:0/grad/hist is illegal; using dense_1/bias_0/grad/hist instead.
INFO:tensorflow:Su

2019-05-21T14:20:26.485787, step: 35, loss: 0.7632007002830505, acc: 0.6406, auc: 0.6863, precision: 0.5926, recall: 0.5714
2019-05-21T14:20:26.679270, step: 36, loss: 1.0213849544525146, acc: 0.5469, auc: 0.5941, precision: 0.5325, recall: 0.6508
2019-05-21T14:20:26.867766, step: 37, loss: 0.8850215673446655, acc: 0.5938, auc: 0.6393, precision: 0.6296, recall: 0.5152
2019-05-21T14:20:27.064269, step: 38, loss: 0.76563960313797, acc: 0.6562, auc: 0.6994, precision: 0.6364, recall: 0.5
2019-05-21T14:20:27.260743, step: 39, loss: 1.0162913799285889, acc: 0.5312, auc: 0.6934, precision: 0.7241, recall: 0.2877
2019-05-21T14:20:27.471153, step: 40, loss: 1.0002048015594482, acc: 0.6328, auc: 0.6562, precision: 0.6353, recall: 0.7714
2019-05-21T14:20:27.663638, step: 41, loss: 0.9229214787483215, acc: 0.5938, auc: 0.6709, precision: 0.5698, recall: 0.7656
2019-05-21T14:20:27.855125, step: 42, loss: 0.8876346349716187, acc: 0.625, auc: 0.6632, precision: 0.6667, recall: 0.5231
2019-05-21T14:

2019-05-21T14:20:47.456709, step: 101, loss: 0.49703481793403625, acc: 0.8125, auc: 0.9091, precision: 0.8913, recall: 0.6833
2019-05-21T14:20:47.655179, step: 102, loss: 0.5653517246246338, acc: 0.8125, auc: 0.871, precision: 0.8814, recall: 0.7536
2019-05-21T14:20:47.853648, step: 103, loss: 0.5030131340026855, acc: 0.8281, auc: 0.9048, precision: 0.8261, recall: 0.8507
2019-05-21T14:20:48.046133, step: 104, loss: 0.3823525905609131, acc: 0.8359, auc: 0.929, precision: 0.8611, recall: 0.8493
2019-05-21T14:20:48.238649, step: 105, loss: 0.5926997661590576, acc: 0.7891, auc: 0.8652, precision: 0.7903, recall: 0.7778
2019-05-21T14:20:48.431104, step: 106, loss: 0.5325844883918762, acc: 0.7812, auc: 0.9312, precision: 0.9231, recall: 0.5902
2019-05-21T14:20:48.624615, step: 107, loss: 0.494932621717453, acc: 0.7812, auc: 0.9023, precision: 0.7778, recall: 0.7778
2019-05-21T14:20:48.814113, step: 108, loss: 0.4166436791419983, acc: 0.8359, auc: 0.9238, precision: 0.8281, recall: 0.8413
20

2019-05-21T14:21:00.613556, step: 167, loss: 0.23260141909122467, acc: 0.9062, auc: 0.9677, precision: 0.88, recall: 0.88
2019-05-21T14:21:00.826986, step: 168, loss: 0.32750940322875977, acc: 0.8828, auc: 0.9521, precision: 0.8636, recall: 0.9048
2019-05-21T14:21:01.032408, step: 169, loss: 0.20751681923866272, acc: 0.9453, auc: 0.9742, precision: 0.9298, recall: 0.9464
2019-05-21T14:21:01.240880, step: 170, loss: 0.31353941559791565, acc: 0.8438, auc: 0.9878, precision: 1.0, recall: 0.7222
2019-05-21T14:21:01.433335, step: 171, loss: 0.24505707621574402, acc: 0.8828, auc: 0.9657, precision: 0.8814, recall: 0.8667
2019-05-21T14:21:01.622858, step: 172, loss: 0.43998441100120544, acc: 0.8438, auc: 0.9648, precision: 0.7722, recall: 0.9683
2019-05-21T14:21:01.818306, step: 173, loss: 0.31682246923446655, acc: 0.8594, auc: 0.9496, precision: 0.8451, recall: 0.8955
2019-05-21T14:21:02.010790, step: 174, loss: 0.5361911058425903, acc: 0.8125, auc: 0.9174, precision: 0.9167, recall: 0.6111


2019-05-21T14:21:21.208530, step: 232, loss: 0.34142979979515076, acc: 0.8828, auc: 0.9462, precision: 0.9286, recall: 0.8667
2019-05-21T14:21:21.402980, step: 233, loss: 0.1815945953130722, acc: 0.9297, auc: 0.9826, precision: 0.9322, recall: 0.9167
2019-05-21T14:21:21.611423, step: 234, loss: 0.34454718232154846, acc: 0.8672, auc: 0.9432, precision: 0.8679, recall: 0.8214
2019-05-21T14:21:21.820863, step: 235, loss: 0.5183038115501404, acc: 0.7734, auc: 0.9036, precision: 0.8182, recall: 0.7031
2019-05-21T14:21:22.032329, step: 236, loss: 0.3657265305519104, acc: 0.8828, auc: 0.9409, precision: 0.8814, recall: 0.8667
2019-05-21T14:21:22.229799, step: 237, loss: 0.22168314456939697, acc: 0.9141, auc: 0.9744, precision: 0.9589, recall: 0.8974
2019-05-21T14:21:22.423252, step: 238, loss: 0.3593842387199402, acc: 0.9219, auc: 0.9533, precision: 0.8889, recall: 0.9697
2019-05-21T14:21:22.637679, step: 239, loss: 0.3857070207595825, acc: 0.8906, auc: 0.9446, precision: 0.8667, recall: 0.89

...

2019-05-21T14:21:55.002164, step: 364, loss: 0.22309333086013794, acc: 0.9531, auc: 0.982, precision: 0.9851, recall: 0.9296
2019-05-21T14:21:55.204592, step: 365, loss: 0.08549501746892929, acc: 0.9609, auc: 0.9944, precision: 0.9559, recall: 0.9701
2019-05-21T14:21:55.405086, step: 366, loss: 0.1551872044801712, acc: 0.9375, auc: 0.9895, precision: 0.9123, recall: 0.9455
2019-05-21T14:21:55.607544, step: 367, loss: 0.1141849234700203, acc: 0.9609, auc: 0.9926, precision: 0.9706, recall: 0.9565
2019-05-21T14:21:55.817951, step: 368, loss: 0.11624563485383987, acc: 0.9453, auc: 0.9931, precision: 0.942, recall: 0.9559
2019-05-21T14:21:56.015423, step: 369, loss: 0.27165913581848145, acc: 0.9297, auc: 0.9721, precision: 0.9524, recall: 0.9091
2019-05-21T14:21:56.210931, step: 370, loss: 0.10924561321735382, acc: 0.9531, auc: 0.9934, precision: 0.95, recall: 0.95
2019-05-21T14:21:56.409370, step: 371, loss: 0.10485701262950897, acc: 0.9375, auc: 0.9933, precision: 0.9608, recall: 0.8909


2019-05-21T14:22:15.970245, step: 429, loss: 0.12483979016542435, acc: 0.9609, auc: 0.9922, precision: 1.0, recall: 0.9265
2019-05-21T14:22:16.166720, step: 430, loss: 0.1907961070537567, acc: 0.9219, auc: 0.9788, precision: 0.9286, recall: 0.9286
2019-05-21T14:22:16.362240, step: 431, loss: 0.1423654705286026, acc: 0.9219, auc: 0.9914, precision: 0.9012, recall: 0.9733
2019-05-21T14:22:16.559701, step: 432, loss: 0.12445050477981567, acc: 0.9609, auc: 0.9895, precision: 0.9412, recall: 0.96
2019-05-21T14:22:16.754180, step: 433, loss: 0.15000450611114502, acc: 0.9453, auc: 0.9884, precision: 0.9636, recall: 0.9138
2019-05-21T14:22:16.951620, step: 434, loss: 0.2172778993844986, acc: 0.9453, auc: 0.9777, precision: 0.9841, recall: 0.9118
2019-05-21T14:22:17.153082, step: 435, loss: 0.09999939799308777, acc: 0.9531, auc: 0.9936, precision: 0.9571, recall: 0.9571
2019-05-21T14:22:17.347593, step: 436, loss: 0.20032484829425812, acc: 0.9375, auc: 0.9809, precision: 0.9538, recall: 0.9254


2019-05-21T14:22:29.021375, step: 495, loss: 0.0854085236787796, acc: 0.9453, auc: 1.0, precision: 1.0, recall: 0.8833
2019-05-21T14:22:29.218846, step: 496, loss: 0.07378601282835007, acc: 0.9766, auc: 0.9993, precision: 1.0, recall: 0.9577
2019-05-21T14:22:29.414323, step: 497, loss: 0.036204949021339417, acc: 0.9844, auc: 0.9993, precision: 1.0, recall: 0.9718
2019-05-21T14:22:29.610800, step: 498, loss: 0.07281112670898438, acc: 0.9375, auc: 0.9956, precision: 0.9672, recall: 0.9077
2019-05-21T14:22:29.803283, step: 499, loss: 0.058760061860084534, acc: 0.9844, auc: 0.9995, precision: 0.9697, recall: 1.0
2019-05-21T14:22:30.009734, step: 500, loss: 0.017608877271413803, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0

Evaluation:
2019-05-21T14:22:37.672212, step: 500, loss: 0.5891967484584222, acc: 0.863992307692308, auc: 0.9416358974358972, precision: 0.839053846153846, recall: 0.9019128205128205
2019-05-21T14:22:37.867689, step: 501, loss: 0.08199064433574677, acc: 0.9688, auc: 0

2019-05-21T14:22:49.760915, step: 561, loss: 0.015103764832019806, acc: 1.0, auc: 1.0, precision: 1.0, recall: 1.0
2019-05-21T14:22:49.960352, step: 562, loss: 0.06726492196321487, acc: 0.9844, auc: 0.9968, precision: 0.9841, recall: 0.9841
2019-05-21T14:22:50.155861, step: 563, loss: 0.0332767628133297, acc: 0.9766, auc: 1.0, precision: 1.0, recall: 0.9444
2019-05-21T14:22:50.356295, step: 564, loss: 0.10883253067731857, acc: 0.9688, auc: 0.9944, precision: 0.9825, recall: 0.9492
2019-05-21T14:22:50.557788, step: 565, loss: 0.05533318594098091, acc: 0.9766, auc: 0.998, precision: 0.9863, recall: 0.973
2019-05-21T14:22:50.753232, step: 566, loss: 0.05400428548455238, acc: 0.9844, auc: 0.9978, precision: 0.9833, recall: 0.9833
2019-05-21T14:22:50.948718, step: 567, loss: 0.0912766233086586, acc: 0.9766, auc: 0.9961, precision: 0.9677, recall: 0.9836
2019-05-21T14:22:51.143219, step: 568, loss: 0.06280166655778885, acc: 0.9766, auc: 0.9985, precision: 1.0, recall: 0.9524
2019-05-21T14:22

2019-05-21T14:23:10.229224, step: 626, loss: 0.054266832768917084, acc: 0.9844, auc: 0.999, precision: 1.0, recall: 0.9667
2019-05-21T14:23:10.423702, step: 627, loss: 0.032248105853796005, acc: 0.9844, auc: 1.0, precision: 1.0, recall: 0.9683
2019-05-21T14:23:10.621176, step: 628, loss: 0.029827618971467018, acc: 0.9766, auc: 1.0, precision: 1.0, recall: 0.9474
2019-05-21T14:23:10.815653, step: 629, loss: 0.02614426240324974, acc: 0.9922, auc: 1.0, precision: 0.9859, recall: 1.0
2019-05-21T14:23:11.014129, step: 630, loss: 0.0415182039141655, acc: 0.9766, auc: 0.9993, precision: 0.9726, recall: 0.9861
2019-05-21T14:23:11.207607, step: 631, loss: 0.033303432166576385, acc: 0.9922, auc: 0.9998, precision: 0.9844, recall: 1.0
2019-05-21T14:23:11.417017, step: 632, loss: 0.061759453266859055, acc: 0.9844, auc: 0.9975, precision: 0.9677, recall: 1.0
2019-05-21T14:23:11.616512, step: 633, loss: 0.02786361426115036, acc: 0.9922, auc: 0.9995, precision: 1.0, recall: 0.9828
2019-05-21T14:23:11